In [1]:
%load_ext watermark
%watermark

Last updated: 2022-01-24T14:19:05.146764-03:00

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.29.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 11, GenuineIntel
CPU cores   : 4
Architecture: 64bit



In [2]:
import pandas as pd
noticias = pd.read_csv("noticias.csv")
noticias.head()

,descripcion,categoria
0,"Aunque parezca mentira, las emisiones de dióxi...",cultura
1,Hubo un proyecto impulsado por la Unión Europe...,cultura
2,China ha confirmado la conclusión con éxito de...,tecnología
3,"En su fructífera carrera como humorista, actor...",cultura
4,Tras dos años de negociación entre la instituc...,cultura


In [3]:
noticias.shape

(16495, 2)

In [4]:
noticias.categoria.value_counts()

cultura       9001
tecnología    4198
ocio          3296
Name: categoria, dtype: int64

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
TfidfVectorizer?

In [7]:
# https://github.com/stopwords-iso/stopwords-es
# import nltk from nltk.corpus import stopwords sw_nltk = stopwords.words('spanish') print(sw_nltk)

In [8]:
import json
with open("stopwords-es.json") as fname:
    stopword_es = json.load(fname)

In [ ]:
stopword_es[:30]

In [10]:
vectorizador = TfidfVectorizer(strip_accents="unicode",stop_words=stopword_es)

In [11]:
vectorizador

TfidfVectorizer(stop_words=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                            '_', 'a', 'actualmente', 'acuerdo', 'adelante',
                            'ademas', 'ademÃ¡s', 'adrede', 'afirmÃ³', 'agregÃ³',
                            'ahi', 'ahora', 'ahÃ\xad', 'al', 'algo', 'alguna',
                            'algunas', 'alguno', 'algunos', 'algÃºn', ...],
                strip_accents='unicode')

In [12]:
ms = vectorizador.fit_transform(noticias.descripcion)

C:\Users\Gamer\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
  warnings.warn('Your stop_word

In [13]:
ms.shape

(16495, 59952)

In [ ]:
# sparse == dispersa
print(ms)

In [ ]:
!pip install mlxtend
# rasbt.github.io/mlxtend

In [17]:
from sklearn.pipeline import make_pipeline
from mlxtend.preprocessing import DenseTransformer

In [18]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [21]:
pipeline_gauss = make_pipeline(vectorizador,DenseTransformer(),GaussianNB())

In [22]:
pipeline_gauss

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(stop_words=['0', '1', '2', '3', '4', '5', '6',
                                             '7', '8', '9', '_', 'a',
                                             'actualmente', 'acuerdo',
                                             'adelante', 'ademas', 'ademÃ¡s',
                                             'adrede', 'afirmÃ³', 'agregÃ³',
                                             'ahi', 'ahora', 'ahÃ\xad', 'al',
                                             'algo', 'alguna', 'algunas',
                                             'alguno', 'algunos', 'algÃºn', ...],
                                 strip_accents='unicode')),
                ('densetransformer', DenseTransformer()),
                ('gaussiannb', GaussianNB())])

In [24]:
# X_train, X_test, y_train, y_test =
pipeline_gauss.fit(X = noticias.descripcion, y = noticias.categoria)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(stop_words=['0', '1', '2', '3', '4', '5', '6',
                                             '7', '8', '9', '_', 'a',
                                             'actualmente', 'acuerdo',
                                             'adelante', 'ademas', 'ademÃ¡s',
                                             'adrede', 'afirmÃ³', 'agregÃ³',
                                             'ahi', 'ahora', 'ahÃ\xad', 'al',
                                             'algo', 'alguna', 'algunas',
                                             'alguno', 'algunos', 'algÃºn', ...],
                                 strip_accents='unicode')),
                ('densetransformer', DenseTransformer()),
                ('gaussiannb', GaussianNB())])

In [25]:
pipeline_gauss.predict(noticias.descripcion)

array(['cultura', 'cultura', 'tecnología', ..., 'cultura', 'tecnología',
       'ocio'], dtype='<U10')

In [27]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [28]:
def f1_curso(modelo, X, y):
    preds = modelo.predict(X)
    return f1_score(y,preds, average="micro")

In [29]:
cross_val_score(pipeline_gauss, noticias.descripcion, noticias.categoria, scoring = f1_curso)

C:\Users\Gamer\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
  warnings.warn('Your stop_word

array([0.62988784, 0.64352834, 0.64807517, 0.63776902, 0.63504092])

In [30]:
pipeline_bernoulli = make_pipeline(TfidfVectorizer(strip_accents="unicode",stop_words=stopword_es,max_features=500)
                               ,DenseTransformer()
                               ,BernoulliNB())

In [31]:
cross_val_score(pipeline_bernoulli,noticias.descripcion,noticias.categoria,scoring=f1_curso)

C:\Users\Gamer\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
  warnings.warn('Your stop_word

array([0.66595938, 0.66110943, 0.66414065, 0.66565626, 0.65807821])

In [32]:
# MultinomialNB
pipeline_multinomial = make_pipeline(TfidfVectorizer(strip_accents="unicode",stop_words=stopword_es,max_features=500)
                               ,DenseTransformer()
                               ,MultinomialNB())

In [33]:
cross_val_score(pipeline_multinomial,noticias.descripcion,noticias.categoria,scoring=f1_curso)

C:\Users\Gamer\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
  warnings.warn('Your stop_word

array([0.68778418, 0.67626554, 0.67262807, 0.68475296, 0.66717187])